# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [3]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [6]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [7]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);

In [8]:
train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,4.55587,0.0,18.1,0,0.718,3.561,87.9,1.6132,24,666
2,0.03537,34.0,6.09,0,0.433,6.59,40.4,5.4917,7,329
3,0.28955,0.0,10.59,0,0.489,5.412,9.8,3.5875,4,277
4,8.24809,0.0,18.1,0,0.713,7.393,99.3,2.4527,24,666
5,0.12204,0.0,2.89,0,0.445,6.625,57.8,3.4952,2,276
6,41.5292,0.0,18.1,0,0.693,5.531,85.4,1.6074,24,666
7,7.52601,0.0,18.1,0,0.713,6.417,98.3,2.185,24,666
8,6.44405,0.0,18.1,0,0.584,6.425,74.8,2.2004,24,666
9,0.07503,33.0,2.18,0,0.472,7.42,71.9,3.0992,7,222


In [9]:
describe(boston)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Nothing,Nothing,DataType
1,Crim,3.61352,0.00632,0.25651,88.9762,,,Float64
2,Zn,11.3636,0.0,0.0,100.0,,,Float64
3,Indus,11.1368,0.46,9.69,27.74,,,Float64
4,Chas,0.06917,0,0.0,1,,,Int64
5,NOx,0.554695,0.385,0.538,0.871,,,Float64
6,Rm,6.28463,3.561,6.2085,8.78,,,Float64
7,Age,68.5749,2.9,77.5,100.0,,,Float64
8,Dis,3.79504,1.1296,3.20745,12.1265,,,Float64
9,Rad,9.54941,1,5.0,24,,,Int64


In [10]:
ols = GLM.lm(@formula(MedV ~ Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

MedV ~ 1 + Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat

Coefficients:
────────────────────────────────────────────────────────────────────────────────────
                Estimate  Std. Error    t value  Pr(>|t|)     Lower 95%    Upper 95%
────────────────────────────────────────────────────────────────────────────────────
(Intercept)   20.2126     5.4132       3.73395     0.0002    9.56997     30.8552    
Crim          -0.0964794  0.0308074   -3.1317      0.0019   -0.157048    -0.0359106 
Zn             0.0345868  0.0136618    2.53165     0.0117    0.00772706   0.0614466 
Indus          0.0488564  0.0594642    0.82161     0.4118   -0.0680532    0.165766  
Chas           1.33934    0.869225     1.54085     0.1242   -0.369598     3.04828   
NOx          -15.1599     4.03208     

In [11]:
predict(ols, test)

101-element Array{Union{Missing, Float64},1}:
  6.766559963121555  
 29.34449433525835   
 13.347490787420838  
 24.077718349240683  
 28.422552996699956  
  7.416988226793267  
 17.172522926392922  
 19.77708744951681   
 35.682650420893665  
 12.335837296693263  
 19.837294513706155  
 13.100292844173474  
 33.8560144227395    
  ⋮                  
 21.99230732794158   
 19.145350807602888  
 16.76155211799942   
 20.084318622499133  
 23.690198193512366  
 22.635044852226436  
 23.70766864276061   
 -0.13091999664428755
 32.842754626810816  
 32.458289656354175  
 29.08434508944555   
 30.053979952599487  

In [12]:
GLM.r²(ols)

0.7936344160181026

In [13]:
GLM.adjr²(ols)

0.7867731561926175